In [1]:
%reset -f
import os
import numpy as np
from numpy import inf
from numpy import nan

from glob import glob

import scipy

import nibabel as nib

import nilearn
import nilearn.plotting


# Create thumbnail images of subject by task and save out to png
# Perform correlation of baseline speech network with neurosynth network
# Annotate with whether null trials were presented (in which case a baseline speech network should be present)

## Known issues:
#  Annotation of null/not null assumes first file returned describes null events
#  For multi session subjects, not clear what the l1 output corresponds to (session1, 2, ??)

## About
#  Greg Ciccarelli
#  Nov 2, 2015

In [2]:
import Image
import ImageFont, ImageDraw

In [3]:
# Dictionary of task and contrast number corresponding to baseline speech network
tasks = sorted(["task001","task002","task003","task005","task006","task007"])
#tasks = sorted(tasks)


In [10]:
img_path = '/om/project/voice/processedData/plots/speech_baseline/cool'
l1dir = '/om/project/voice/processedData/l1analysis/l1output_20151202'
model = 'model200'

In [5]:
# Neurosynth speech network
speech_img = nib.load('/om/user/mathiasg/projects/voice/speechproduction_pAgF_z_FDR_0.01.nii.gz')
speech_mat = np.array(speech_img.get_data())
speech_vec = np.copy(speech_mat)
speech_vec = np.reshape(speech_vec, (np.size(speech_vec),1))

In [6]:
imgs = os.listdir(img_path)
imgs = sorted(imgs)

In [8]:
subjs = []    
for sub in imgs:
    if sub[:8] not in subjs:
        subjs.append(sub[:8])

In [9]:
outliers = np.genfromtxt("outliers_voice.txt",dtype=str)

def countMotion(s,t):
    count = 0
    for x in outliers:
        if s in x and t in x:
            count += 1
    return count

In [10]:
def getRho(subj,task):
    
    zpath = os.path.join(l1dir, model, task, model, task, 
                            subj, 'zstats', 'mni', 'zstat' + '01.nii.gz')
    # Check if image
    if os.path.exists(zpath):
        zstat = nib.load(zpath)
        zstat_mat = np.array(zstat.get_data())
        zstat_vec = np.copy(zstat_mat)
        zstat_vec = np.reshape(zstat_vec, (np.size(zstat_vec),1))  
                
        [rho, p] = scipy.stats.pearsonr(speech_vec, zstat_vec)
        format(float(rho),'.3f')
                    
    else:
        
        rho = -1
    return rho

In [40]:
# Assume first file is representative for the subject- 
# not true under unusual circumstance of multi session subject
def checkNulls(subj,task):
    hasNull = False
    file_path_psylog = glob(os.path.join('/om/project/voice/processedData/audio/psycholog/',
                                           task, subj, 'session00*/R00*/*.tsv'))
    
    if file_path_psylog:
        file_path_psylog= file_path_psylog[0]
        data = np.genfromtxt(file_path_psylog, delimiter='\t', dtype=str)
        if 'NULL' in data[:,2] or 'null' in data[:,2]:
            hasNull = True
    return hasNull

In [46]:
def plot_subj_by_task(subjs, tasks, img_path, file_name_save): #, speech_vec):
    
    ## Input
    
    ## Output
    #  *.png
    
    # NOTE:  Image module transposes row and column dimensions
    
    N_px_row = 400
    N_px_col = 600
    shiftx = 90
    shifty = 100
    
    N_row = len(subjs)
    N_col = len(tasks) #*2 #Times 2 for lh and rh
    #creates a new empty image, RGB mode, and size 400 by 400.
    new_im = Image.new('RGB', (N_col*N_px_col + shiftx, N_row*N_px_row + shifty))
    
    for idx_subj, subj in enumerate(subjs):
        #
        print "=="
        print subj

        for idx_task, task in enumerate(tasks):
            print idx_task
            print task
            
            plot_img = os.path.join(img_path, '%s_%s.png' % (subj,task))
            
            #if task == tasks[-1] and subj == subjs[-1]: ##add color bar for last image
                #plot_img = '/om/project/voice/processedData/plots/speech_baseline/Archive' + \
                 #          '/std_displaythresh/%s_%s.png' % (subj,task)
            
            # Check for image
            if os.path.exists(plot_img):
            
                #opens an image of brain:
                im = Image.open(plot_img)
                #resize so it is no bigger than 100,100
                im.thumbnail((N_px_col,N_px_row))
                #paste the image at location i,j:
                new_im.paste(im, (idx_task*N_px_col + shiftx,idx_subj*N_px_row + shifty))
                draw = ImageDraw.Draw(new_im)
                #font = ImageFont.truetype("arial.ttf", 15)
                #font = ImageFont.load("arial.pil")
                #draw.text((idx_task*N_px_col+20,idx_subj*N_px_row+70), 
                 #         os.path.split(plot_img)[1] + "       " + message.upper())
                
                
                #check motion outliers ====================================================
                motion = countMotion(subj,task)
                if motion == 0:
                    color = Image.open("green.png")
                    message = "No motion outliers"
                elif motion == 1:
                    color = Image.open("yellow.png")
                    message = "1 outlier - run either >0.10 or missing"
                elif motion >= 2:
                    color = Image.open("red.png")
                    message = "2 or more outliers"
                else:
                    continue
                color.thumbnail((N_px_col/11,N_px_row))
                new_im.paste(color, (idx_task*N_px_col+40, idx_subj*N_px_row+shifty))
                draw = ImageDraw.Draw(new_im)
                draw.text((idx_task*N_px_col+50, idx_subj*N_px_row+shifty+20), 
                          "motion", fill=(0,0,0,0))
                
                # label participant and print message
                draw.text((idx_task*N_px_col+20,idx_subj*N_px_row+70), 
                          os.path.split(plot_img)[1] + "       " + message.upper())

            
            
            
                #neurosynth corr ====================================================
                rho = getRho(subj,task)
                
                if rho >= 0.15:
                    color = Image.open("green.png")
                elif rho == -1:
                    color = Image.open("red.png")
                else:
                    color = Image.open("yellow.png")
                color.thumbnail((N_px_col/11,N_px_row))
                new_im.paste(color, (idx_task*N_px_col+40, idx_subj*N_px_row+shifty+80))
                draw = ImageDraw.Draw(new_im)
                #put rho in square
                draw.text((idx_task*N_px_col+50, idx_subj*N_px_row+shifty+100), 
                          str(round(rho,3)), fill=(0,0,0,0))
                
                #null trials ====================================================
                hasNull = checkNulls(subj,task)
                if hasNull:
                    color = Image.open("green.png")
                else:
                    color = Image.open("red.png")
                color.thumbnail((N_px_col/11,N_px_row))
                new_im.paste(color, (idx_task*N_px_col+40, idx_subj*N_px_row+shifty+160))
                draw = ImageDraw.Draw(new_im)
                draw.text((idx_task*N_px_col+55, idx_subj*N_px_row+shifty+180), 
                          "null", fill=(0,0,0,0))
            
            
            # if img not found - why?
            else:
                draw = ImageDraw.Draw(new_im)
                draw.text((idx_task*N_px_col+200,idx_subj*N_px_row+300), 
                          "missing %s %s" % (subj,task), fill=(255,0,0,255))
                

                
    #new_im.show()
    new_im.save(file_name_save)                

    #f.savefig(file_name_save)  
    #plt.close(f)      

In [21]:
# Tests
rho = 0.1546
#str(round(rho,3))
str(format(rho,'.3f'))
#str(format(round((rho,3)),'.3f'))

'0.155'

In [12]:
subs8 = []
for sub in subjs:
    if sub[:6] == 'voice8':
        subs8.append(sub)

subs9 = []
for sub in subjs:
    if sub[:6] == 'voice9':
        subs9.append(sub)

In [47]:
# save 800s
file_name_save = '20160108_voice8xx.png'
plot_subj_by_task(subs8, tasks, img_path, file_name_save)

==
voice854
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice862
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice864
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice867
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice872
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice873
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice875
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice877
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice880
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice884
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice886
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice889
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice891
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice893
0
task001
1
task002
2
task003
3
task005
4
task006
5


In [48]:
# save 900s
file_name_save = '20160108_voice9xx.png'
plot_subj_by_task(subs9, tasks, img_path, file_name_save)

==
voice973
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice974
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice975
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice976
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice978
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice979
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice980
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice981
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice982
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice983
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice984
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice986
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice987
0
task001
1
task002
2
task003
3
task005
4
task006
5
task007
==
voice988
0
task001
1
task002
2
task003
3
task005
4
task006
5


In [ ]:
----------
-

-
-
-
--
-
-
--

-
--
-
-
--

----

--

In [ ]:
def plot_subj_by_task(subjs, tasks, img_path, file_name_save): #, speech_vec):
    
    ## Input
    
    ## Output
    #  *.png
    
    # NOTE:  Image module transposes row and column dimensions
    
    N_px_row = 400
    N_px_col = 200
    shift = 25
    
    N_row = len(subjs)
    N_col = len(tasks)*2 # Times 2 for lh and rh
    #creates a new empty image, RGB mode, and size 400 by 400.
    new_im = Image.new('RGB', (N_col*N_px_col, N_row*N_px_row + shift))
    
    for idx_subj, subj in enumerate(subjs):
        #
        print "=="
        print subj

        for idx_task, task in enumerate(tasks):
            print idx_task
            print task
            print task_contrast[task]
            
            #==========================================================================
            file_path_data = os.path.join(file_path_l1.replace('_png', ''), task, subj, 
                                          'zstats', 'mni', 'zstat' + task_contrast[task] + '.nii.gz')
            print file_path_data
            # Check if image
            if os.path.exists(file_path_data):
                zstat = nib.load(file_path_data)
                zstat_mat = np.array(zstat.get_data())
                zstat_vec = np.copy(zstat_mat)
                zstat_vec = np.reshape(zstat_vec, (np.size(zstat_vec),1))  
                
                [rho, p] = scipy.stats.pearsonr(speech_vec, zstat_vec)
            else:
                rho = 2
                
                
            #==========================================================================
            # Check if null trial
            # Assume first file is representative for the subject- 
            # not true under unusual circumstance of multi session subject
            file_path_psylog = glob(os.path.join('/om/project/voice/processedData/audio/psycholog/',
                                            task, subj, 'session00*/R00*/*.tsv'))
            if file_path_psylog:
                file_path_psylog= file_path_psylog[0]
                data = np.genfromtxt(file_path_psylog, delimiter='\t', dtype=str)
                if 'NULL' in data[:,2]:
                    null_str = 'Null'
                else:
                    null_str = 'No Null'
            else:
                null_str = 'Unk'            
                
            #---------------------------------------------------------------------------
            # Check if dicom converted data
            nifti_present = glob(os.path.join(
                '/om/project/voice/processedData/openfmri', subj, 
                    'session*', 'functional', subj + '_' + task + '*.nii.gz'))

            #---------------------------------------------------------------------------
            hemi_str = 'rh'
            file_path_data = os.path.join(img_path, '%s_%s.png' % (subj,task))
            
            # Check if image
            if os.path.exists(file_path_data):
                #opens an image:
                im = Image.open(file_path_data)
                #resize so it is no bigger than 100,100
                im.thumbnail((N_px_col,N_px_row))
                #paste the image at location i,j:
                new_im.paste(im, (idx_task*N_px_col*2,idx_subj*N_px_row + shift))
                draw = ImageDraw.Draw(new_im)
                #font = ImageFont.truetype("arial.ttf", 15)
                #font = ImageFont.load("arial.pil")
                draw.text((idx_task*N_px_col*2,idx_subj*N_px_row), 
                          os.path.split(file_path_data)[1])
            elif nifti_present:
                #opens an image:
                im = Image.open('/om/user/gr21783/git/voice/mri/scripts/analysis_openfmri/dataQuality/yellow.png')
                #resize so it is no bigger than 100,100
                im.thumbnail((N_px_col,N_px_row))
                #paste the image at location i,j:
                new_im.paste(im, (idx_task*N_px_col*2,idx_subj*N_px_row + shift))
                draw = ImageDraw.Draw(new_im)                
                draw.text((idx_task*N_px_col*2,idx_subj*N_px_row), 
                          os.path.split(file_path_data)[1])
                
            file_path_data = file_path_data.replace('rh', 'lh')
            if os.path.exists(file_path_data):
                im = Image.open(file_path_data)
                im.thumbnail((N_px_col,N_px_row))
                new_im.paste(im, (idx_task*N_px_col*2 + N_px_col,idx_subj*N_px_row + shift))
                #im.show()
                draw = ImageDraw.Draw(new_im)
                #font = ImageFont.truetype("arial.ttf", 15) #doesn't work
                #font = ImageFont.load("arial.pil") #doesn't work
                #draw.text((idx_task*N_px_col*2 + N_px_col,idx_subj*N_px_row), 
                #          os.path.split(file_path_data)[1])   
                draw.text((idx_task*N_px_col*2 + N_px_col,idx_subj*N_px_row), 
                          null_str + " rho: %2.2f" % (rho))
            elif nifti_present:
                #opens an image:
                im = Image.open('/om/user/gr21783/git/voice/mri/scripts/analysis_openfmri/dataQuality/yellow.png')
                #resize so it is no bigger than 100,100
                im.thumbnail((N_px_col,N_px_row))
                #paste the image at location i,j:
                new_im.paste(im, (idx_task*N_px_col*2 + N_px_col,idx_subj*N_px_row + shift))
                draw = ImageDraw.Draw(new_im)                
                draw.text((idx_task*N_px_col*2 + N_px_col,idx_subj*N_px_row), 
                          os.path.split(file_path_data)[1])                
                
            else:
                print "missing " + file_path_data

                
    #new_im.show()
    new_im.save(file_name_save)                

    #f.savefig(file_name_save)  
    #plt.close(f)      